In [64]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import heapq
import time
import bisect
import random
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/Leonardomotta/Lei_de_zipf/master/results.csv') 
                                            

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leonardo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ranqueamento das palavras retiradas do site  https://www.gazetadopovo.com.br/ gerando um dicionario.

In [65]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.texto:
  words = [word for word in tknz.tokenize(str(text).lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

### 1. Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?

In [66]:
queries = ["bolsonaro", "educação", "ministério", "brasil", "economia"]

def n_docs_word(word, ranking):
  return len(ranking[word])

def n_docs_both(a, b, ranking):
  docs_a = [doc for doc,score in ranking[a]]
  docs_b = [doc for doc,score in ranking[b]]
  n = 0
  
  for doc in docs_a:
    if doc in docs_b:
      n += 1
      
  return n

In [67]:
def calc_mim(a, b, ranking):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  mim = n_ab / quo if quo != 0 else 0
  return (mim,b)
  
def calc_emim(a, b, ranking, n):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  exp = n * (n_ab / quo) if quo != 0 else 0
  emim = n_ab * np.log10(exp) if exp != 0 else 0
  return (emim, b)
  
def calc_x(a, b, ranking, n):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  quo = (n_a * n_b)
  x = (n_ab - (1 / n) * n_a * n_b) ** 2 / quo if quo != 0 else 0
  return (x,b)
  
def calc_dice(a, b, ranking):
  n_a = n_docs_word(a, ranking)
  n_b = n_docs_word(b, ranking)
  n_ab = n_docs_both(a, b, ranking)
  dice = n_ab / (n_a + n_b)
  return (dice,b)

In [68]:
def create_metrics_table(query, ranking, n):
  top_10_table = pd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for word in ranking:
    if word != query:
      bisect.insort(MIM, calc_mim(query, word, ranking))
      bisect.insort(EMIM,calc_emim(query, word, ranking, n))
      bisect.insort(X,calc_x(query, word, ranking, n))
      bisect.insort(Dice,calc_dice(query, word, ranking))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  top_10_table['MIM'] = [word[1] for word in MIM[:10]]
  top_10_table['EMIM'] = [word[1] for word in EMIM[:10]]
  top_10_table['X²'] = [word[1] for word in X[:10]]
  top_10_table['Dice'] = [word[1] for word in Dice[:10]]
  
  top_10_table.index += 1
  
  return top_10_table

In [69]:
n = result.texto.count()
table_1 = create_metrics_table(queries[0], indexes, n)
table_2 = create_metrics_table(queries[1], indexes, n)
table_3 = create_metrics_table(queries[2], indexes, n)
table_4 = create_metrics_table(queries[3], indexes, n)
table_5 = create_metrics_table(queries[4], indexes, n)

1.1 Top-10 palavras associadas a 'Bolsonaro' 

In [70]:
table_1

,MIM,EMIM,X²,Dice
1,ótimo,jair,nan,jair
2,ótima,psl,jair,presidente
3,órfão,presidente,psl,governo
4,óbvio,congresso,congresso,psl
5,ético,ministro,deputados,ministro
6,árduo,governo,câmara,disse
7,árabe,câmara,proposta,afirmou
8,ágil,disse,planalto,congresso
9,áfrica,afirmou,previdência,câmara
10,zimbábue,previdência,ministro,sobre


#### 1.2 Top-10 palavras associadas a 'Educação'

In [71]:
table_2

,MIM,EMIM,X²,Dice
1,ético,mec,mec,mec
2,zimmermann,vélez,vélez,vélez
3,zimbábue,indicou,indicou,indicou
4,zanini,reclamar,reclamar,ensino
5,xixi,criticado,criticado,reclamar
6,william,ensino,nan,criticado
7,walter,vou,rodríguez,irá
8,wajngarten,irá,colombiano,ministério
9,waack,escolas,vou,meses
10,vélez,rodríguez,ensino,vou


#### 1.3 Top-10 palavras associadas a 'Ministério'

In [72]:
table_3

,MIM,EMIM,X²,Dice
1,ético,público,nan,ministro
2,éticas,federal,pedir,público
3,árduo,ministro,negou,federal
4,xixi,pedir,mec,fim
5,walter,fim,jornalistas,sido
6,waldir,jornalistas,viagem,justiça
7,wajngarten,mec,reunir,bolsonaro
8,vírus,tribunal,procuradoria,tribunal
9,vizinhas,negou,criticado,ainda
10,vivo,justiça,criminal,feira


#### 1.4 Top-10 palavras associadas a 'Brasil'

In [73]:
table_4

,MIM,EMIM,X²,Dice
1,íngremes,estados,nan,estados
2,ínfima,unidos,estados,país
3,ético,brasileira,brasileira,governo
4,ânimo,país,unidos,presidente
5,ásia,outros,mourão,outros
6,árabe,nacional,hamilton,parte
7,ágil,nível,mudanças,ser
8,zulia,algo,distribuição,sobre
9,zimmermann,parte,algo,ainda
10,zarlenga,países,ministros,ter


#### 1.5 Top-10 palavras associadas a 'Economia'

In [74]:
table_5

,MIM,EMIM,X²,Dice
1,úrzua,guedes,nan,guedes
2,ético,ministro,guedes,ministro
3,álvarez,país,vice,país
4,zimbábue,previdência,mal,bolsonaro
5,zarlenga,críticas,representantes,presidente
6,yuool,vice,críticas,previdência
7,xixi,bolsonaro,vivem,críticas
8,wyllys,presidente,tributária,reforma
9,watch,proposta,luciano,vice
10,washignton,reforma,imposto,proposta



as metricas EMIM, X² e dice possuem resultados bastante similares e satisfatorios , o MIM por sua vez tem resultados bem descrepantes ,  isso é causado porque os valores retornado pela métrica é exatamente o mesmo para uma série de palavras e por isso o algoritmo de montagem das tabelas não consegue identificar com precisão qual as palavras que realmente estão próximas da palavra consultada , acredito que o Dice deva ser a que mais se destaca , por resultados precisos e calculo simples.


### 2. De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.

#### Algortimo : Termo-por-vez


In [75]:
def term_a_time(query, indexes, k):    
    a = {} # Accumulator HashTable
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for index in query_indexes:
      for i in index:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    return heapq.nlargest(k, rank)

In [76]:
def top_docs(query, k):
  
  top_docs = [doc for score,doc in term_a_time(query, indexes, k)]
  top_docs_ranking = {} 
  
  for word in indexes:
    top_docs_ranking[word] = []
    for doc_score in indexes[word]:
      if doc_score[0] in top_docs:
        top_docs_ranking[word].append(doc_score)
        
  return top_docs_ranking

#### Top-3 Documentos

In [77]:
k = 3
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [78]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-3 Documentos de 'Bolsonaro'

In [79]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,jair,área
2,presidente,vou
3,governo,vida
4,psl,vai
5,ministro,tudo
6,disse,tipo
7,afirmou,ter
8,congresso,tempo
9,câmara,sobre
10,sobre,ser


##### Top-10 palavras associadas aos Top-3 Documentos de 'Educação'

In [80]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,mec,nacional
2,vélez,governo
3,indicou,escola
4,ensino,ensino
5,reclamar,durante
6,criticado,índices
7,irá,área
8,ministério,universidades
9,meses,trabalho
10,vou,trabalhadores


##### Top-10 palavras associadas aos Top-3 Documentos de 'Ministério'

In [81]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,ministro,sobre
2,público,sido
3,federal,governo
4,fim,disse
5,sido,desde
6,justiça,contra
7,bolsonaro,vélez
8,tribunal,vou
9,ainda,vida
10,feira,vai


##### Top-10 palavras associadas aos Top-3 Documentos de 'Brasil'

In [82]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,estados,segundo
2,país,país
3,governo,econ
4,presidente,votado
5,outros,vez
6,parte,vai
7,ser,unidos
8,sobre,têm
9,ainda,três
10,ter,trazer


##### Top-10 palavras associadas aos Top-3 Documentos de 'Economia'

In [83]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,guedes,vai
2,ministro,ser
3,país,novo
4,bolsonaro,menos
5,presidente,investimentos
6,previdência,hora
7,críticas,chegou
8,reforma,brasil
9,vice,ainda
10,proposta,três


#### Top-5 Documentos

Expansão de consulta para os top-5 documentos das consultas selecionadas inicialmente.

In [84]:
k = 5
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [85]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-5 Documentos de 'Bolsonaro'

In [86]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,jair,ter
2,presidente,sobre
3,governo,reforma
4,psl,questão
5,ministro,previdência
6,disse,presidente
7,afirmou,paulo
8,congresso,partidos
9,câmara,ministro
10,sobre,militar


##### Top-10 palavras associadas aos Top-5 Documentos de 'Educação'

In [87]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,mec,governo
2,vélez,vai
3,indicou,ser
4,ensino,sendo
5,reclamar,semana
6,criticado,secretaria
7,irá,presidente
8,ministério,pessoas
9,meses,parte
10,vou,nacional


##### Top-10 palavras associadas aos Top-5 Documentos de 'Ministério'

In [88]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,ministro,governo
2,público,sobre
3,federal,ministro
4,fim,desde
5,sido,contra
6,justiça,sido
7,bolsonaro,segurança
8,tribunal,segundo
9,ainda,responsável
10,feira,público


##### Top-10 palavras associadas aos Top-5 Documentos de 'Brasil'

In [89]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,estados,segundo
2,país,país
3,governo,vez
4,presidente,unidos
5,outros,três
6,parte,sobre
7,ser,presidente
8,sobre,nova
9,ainda,mundo
10,ter,maiores


##### Top-10 palavras associadas aos Top-5 Documentos de 'Economia'

In [90]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,guedes,brasil
2,ministro,segundo
3,país,primeiro
4,bolsonaro,outros
5,presidente,ainda
6,previdência,vai
7,críticas,três
8,reforma,tempo
9,vice,sul
10,proposta,sobre


#### Top-10 Documentos

Expansão de consulta para os top-10 documentos das consultas selecionadas inicialmente.

In [91]:
k = 10
ranking_docs1 = top_docs(queries[0], k)
ranking_docs2 = top_docs(queries[1], k)
ranking_docs3 = top_docs(queries[2], k)
ranking_docs4 = top_docs(queries[3], k)
ranking_docs5 = top_docs(queries[4], k)

In [92]:
table_top1 = create_metrics_table(queries[0], ranking_docs1, k)
table_top2 = create_metrics_table(queries[1], ranking_docs2, k)
table_top3 = create_metrics_table(queries[2], ranking_docs3, k)
table_top4 = create_metrics_table(queries[3], ranking_docs4, k)
table_top5 = create_metrics_table(queries[4], ranking_docs5, k)

##### Top-10 palavras associadas aos Top-10 Documentos de 'Bolsonaro'

In [93]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_1['Dice']
table_dice['Dice after Term A Time'] = table_top1['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,jair,presidente
2,presidente,governo
3,governo,brasil
4,psl,sobre
5,ministro,ainda
6,disse,ter
7,afirmou,ser
8,congresso,segundo
9,câmara,reforma
10,sobre,questão


##### Top-10 palavras associadas aos Top-10 Documentos de 'Educação'

In [94]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_2['Dice']
table_dice['Dice after Term A Time'] = table_top2['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,mec,governo
2,vélez,presidente
3,indicou,ainda
4,ensino,vai
5,reclamar,sido
6,criticado,ser
7,irá,pessoas
8,ministério,anos
9,meses,área
10,vou,sobre


##### Top-10 palavras associadas aos Top-10 Documentos de 'Ministério'

In [95]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_3['Dice']
table_dice['Dice after Term A Time'] = table_top3['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,ministro,governo
2,público,desde
3,federal,sobre
4,fim,sido
5,sido,ser
6,justiça,público
7,bolsonaro,ministro
8,tribunal,dois
9,ainda,contra
10,feira,brasileiro


##### Top-10 palavras associadas aos Top-10 Documentos de 'Brasil'

In [96]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_4['Dice']
table_dice['Dice after Term A Time'] = table_top4['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,estados,país
2,país,sobre
3,governo,segundo
4,presidente,presidente
5,outros,vai
6,parte,governo
7,ser,disse
8,sobre,ainda
9,ainda,unidos
10,ter,três


##### Top-10 palavras associadas aos Top-10 Documentos de 'Economia'

In [97]:
table_dice = pd.DataFrame()
table_dice['Dice before Term A Time'] = table_5['Dice']
table_dice['Dice after Term A Time'] = table_top5['Dice']
table_dice

,Dice before Term A Time,Dice after Term A Time
1,guedes,ainda
2,ministro,presidente
3,país,brasil
4,bolsonaro,sobre
5,presidente,ser
6,previdência,segundo
7,críticas,parte
8,reforma,outros
9,vice,ministério
10,proposta,menos


#### Resultado:

Aumentando o k os resultados possuem maior qualidade , isso deve se ao fato de que termos um maior numero de dados a respeito da ligação de palavras ,  comparando o top 3 e o top 10 , vemos que a diferença de qualidade é gritante , porem com top-k docs para expansão , aumenta bastante a agilidade para computar palavras mais relacionadas a consulta.
